In [1]:
# import stuff
%load_ext autoreload
%autoreload 2
%matplotlib inline

from random import randint
import matplotlib

import numpy as np
import torch

GLOVE_PATH = '../dataset/GloVe/glove.840B.300d.txt'

In [2]:
model = torch.load('infersent.allnli.pickle', map_location=lambda storage, loc: storage)
# On CPU, setting the right number of threads with "torch.set_num_threads(k)" may improve performance

In [3]:
model.set_glove_path(GLOVE_PATH)

In [4]:
model.build_vocab_k_words(K=100000)

Vocab size : 100000


# Load sentences

In [5]:
import string
import re
from nltk.tokenize import sent_tokenize
punct = set(string.punctuation)

In [6]:
f = open('hotel_218524.dat', 'r', encoding='utf-8', errors='replace')
sentences = []
for i in f:
    if re.match('<Content>.*', i) and len(i)>150:
        temp = str(i[9:len(i)-2])
        temp = ''.join(x for x in temp)
        sentences.append(temp)

In [7]:
print(sentences)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [8]:
print(sentences[:2])

["Perfect Honeymoon!!!! We had the most wonderful time at the Excellence. We opted for the Excellence Club and it was well worht it for the perks. Roberto Puello who was the Club Manager did an extra special job of making us feel welcomed! We took some great pictures that looked like postcards. The food was excellent the Pizza was close to being our favorite. The shows were great especially the Michael Jackson and the 70's  80's night. The French resturant was probably our favorite. We will definately be back for an anniversary trip. We made some friends but had plenty of privcacy as well. The rooms were great and everytime we left and came back it was cleaned, towels restocked and the bar was as well!", 'A truly fantastic place Xmas  We have just returned from a 2 week Xmas  New Year break at the Excellence Punta Cana. The hotel is magnificent and we have never experienced such incredible service before. The restaurants are great, with plenty of choice. The whole complex is so good th

In [9]:
sentences_new = []
for i in sentences:
    sentences_new.extend(sent_tokenize(i))

In [10]:
print(sentences_new[:2])

['Perfect Honeymoon!!!!', 'We had the most wonderful time at the Excellence.']


# Encode sentences

In [11]:
embeddings = model.encode(sentences_new, bsize=128, tokenize=False, verbose=True)
print('nb sentences encoded : {0}'.format(len(embeddings)))

Nb words kept : 965026/1093750 (88.23 %)
Speed : 48.42 sentences/s (cpu mode, bsize=128)
nb sentences encoded : 58997


In [12]:
print(embeddings.shape)

(58997, 4096)


In [14]:
import pickle
with open('hotel_embeddings.pickle', 'wb') as handle:
    pickle.dump(embeddings, handle)

# Clustering (Spectral)

In [ ]:
from sklearn.cluster import SpectralClustering
from sklearn import metrics

# Spectral Clustering
sc = SpectralClustering(50, affinity='nearest_neighbors', n_init=1000)
sc.fit(embeddings)
print('spectral clustering')
print(sc.labels_)